# Libraries

In [1]:
import os
import sys
import pickle
import datetime, time
import random
from importlib import reload
from math import sqrt
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

module_paths = {os.pardir}  # package directory
sys.path.extend(module_paths.difference(sys.path))


from nytf import utils; reload(utils)
#from nytf import main_pipeline; reload(main_pipeline)
pass

# Loading dataframes

In [2]:
%%time
#train = utils.load_dataframe('train', cloud=False)
#train = pd.concat((train, utils.BasicTemporalFeatures().transform(train)), axis=1)

with open(os.path.join(utils.PROCESSING_DIRECTORY, 'train_basic_features.pkl'), 'rb') as f:
    train = pickle.load(f)

test = utils.load_dataframe('test', cloud=False)

Wall time: 26.9 s


In [3]:
train.memory_usage().sum()/10**9  # In Go

4.37622669

# RandomForest

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [5]:
%%time
N = 500_000
ind = sorted(random.sample(range(train.shape[0]), N))
X = train.iloc[ind][['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
           'timestamp', 'week_progress_cos', 'week_progress_sin', 'month_progress_cos', 'month_progress_sin',
           'year_progress_cos', 'year_progress_sin', 'day_progress_cos', 'day_progress_sin']].values
y = train.iloc[ind]['fare_amount'].values

Wall time: 6.06 s


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.5)

d:\Profiles\thaugomat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
regr = RandomForestRegressor()

In [8]:
%%time
regr.fit(X_train, y_train)

d:\Profiles\thaugomat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 1min 36s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [9]:
%%time
y_train_est = regr.predict(X_train).clip(0, 100)
print('Train error :', sqrt(((y_train_est - y_train) ** 2).mean()))
y_test_est = regr.predict(X_test).clip(0, 100)
print('Test error :', sqrt(((y_test_est - y_test) ** 2).mean()))

Train error : 1.5712010301534194
Test error : 3.734254569424527
Wall time: 4.13 s


# NeuralNetwork